<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Les-imports-et-initialisations-des-variables" data-toc-modified-id="Les-imports-et-initialisations-des-variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Les imports et initialisations des variables</a></span></li><li><span><a href="#Les-fonctions-analytiques" data-toc-modified-id="Les-fonctions-analytiques-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Les fonctions analytiques</a></span><ul class="toc-item"><li><span><a href="#La-clause-partition-by" data-toc-modified-id="La-clause-partition-by-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>La clause partition by</a></span></li><li><span><a href="#La-clause-order-by" data-toc-modified-id="La-clause-order-by-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>La clause order by</a></span></li><li><span><a href="#Le-palmarès" data-toc-modified-id="Le-palmarès-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Le palmarès</a></span></li><li><span><a href="#Le-palmarès-(dense_rank)" data-toc-modified-id="Le-palmarès-(dense_rank)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Le palmarès (dense_rank)</a></span></li><li><span><a href="#La-clause-de-fenêtrage" data-toc-modified-id="La-clause-de-fenêtrage-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>La clause de fenêtrage</a></span></li></ul></li></ul></div>

# Les imports et initialisations des variables

In [ ]:
import pandas as pd 
import seaborn as sns
import warnings 
import os
import cx_Oracle
import pickle
import re
import unicodedata

from datetime import datetime
warnings.filterwarnings(action="ignore")

%pylab inline
%load_ext sql

plt.style.use('seaborn-darkgrid')
sns.set(font_scale=3)

In [ ]:
sys.path.append(os.path.abspath('../../.'))
from dbconnect import utilisateur,connexion,password
%sql oracle+cx_oracle://$utilisateur:$password@$connexion
connection = cx_Oracle.connect(utilisateur, password, connexion)        

# Les fonctions analytiques 

In [ ]:
%sql select annee, trimestre, \
            sum(port) as port , \
            sum(sum(port)) over (partition by annee) st_a ,\
            sum(sum(port)) over (partition by trimestre) st_t ,\
            sum(sum(port)) over () st_p \
     from commandes \
     group by  annee, trimestre \
     order  by  annee, trimestre 

In [ ]:
query = "select * from commandes"
df = pd.read_sql_query(query, connection) #df.to_sql('nom', con=engine,if_exists='replace')
df

In [ ]:
df.columns

In [ ]:
df1 = df.groupby(['annee','trimestre']).agg({'port':'sum'})

In [ ]:
df1.head()

In [ ]:
df.head()

In [ ]:
df['st_a'] = df.groupby('annee').port.transform('sum')

In [ ]:
df.head()

In [ ]:
df1['st_a'] = df1.groupby('annee').port.transform('sum')
df1['st_t'] = df1.groupby('trimestre').port.transform('sum')
df1['st_p'] = df1.port.sum()

In [ ]:
df1

## La clause partition by

In [ ]:
query = """
SELECT
    co.annee, 
    co.mois,
    cl.pays,
    cl.ville,
    ca.nom_categorie,
    pr.nom_produit,
    sum(dc.prix_unitaire*dc.quantite) ca,
    sum(dc.quantite) quantite,
    sum(dc.remise) remise
FROM
    clients cl
    JOIN commandes co ON cl.code_client = co.code_client
    JOIN details_commandes dc ON co.no_commande = dc.no_commande
    JOIN produits pr ON dc.ref_produit = pr.ref_produit
    JOIN categories ca ON ca.code_categorie = pr.code_categorie
group by co.annee, 
    co.mois,
    ca.nom_categorie,
    pr.nom_produit,
    cl.pays,
    cl.ville
"""
donnees = pd.read_sql_query(query, connection) #df.to_sql('nom', con=engine,if_exists='replace')
donnees

In [ ]:
donnees.columns

In [ ]:
donnees['CA_a'] = donnees.groupby(['annee']).ca.transform('sum')
donnees['CA_am'] = donnees.groupby(['annee','mois']).ca.transform('sum')
donnees['CA_ap'] = donnees.groupby(['annee','pays']).ca.transform('sum')
donnees['CA_ac'] = donnees.groupby(['annee','nom_categorie']).ca.transform('sum')
donnees['CA_apc'] = donnees.groupby(['annee','pays','nom_categorie']).ca.transform('sum')

In [ ]:
donnees.sort_values(['annee','mois','pays','nom_categorie']).head(20)

## La clause order by 

In [ ]:
donnees01 = donnees.groupby(['annee','mois','pays','nom_categorie']).agg({'ca':'sum'})

In [ ]:
donnees01['CA_ca']   = donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).groupby(['annee']).ca.transform('cumsum')
donnees01['CA_cam']  = donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).groupby(['annee','mois']).ca.transform('cumsum')
donnees01['CA_cap']  = donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).groupby(['annee','pays']).ca.transform('cumsum')
donnees01['CA_cac']  = donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).groupby(['annee','nom_categorie']).ca.transform('cumsum')
donnees01['CA_capc'] = donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).groupby(['annee','pays','nom_categorie']).ca.transform('cumsum')

In [ ]:
donnees01.sort_values(['annee','mois','pays','nom_categorie','ca']).head(50)

## Le palmarès

In [ ]:
donnees01 = donnees.groupby(['annee','mois']).agg({'ca':'sum'})

In [ ]:
donnees01.head()

In [ ]:
donnees01['CA_a']        = donnees01.groupby(['annee']).ca.transform('sum')
donnees01['palmares_A']  = donnees01.groupby(['annee']).ca.rank(method='dense')
donnees01['palmares']    = donnees01.ca.rank(method='dense')

In [ ]:
donnees01.head()

In [ ]:
donnees01.sort_values(['annee','palmares_A']).head(10)

In [ ]:
donnees01.sort_values(['palmares_A']).head(10)

In [ ]:
donnees01.sort_values(['palmares']).head(20)

In [ ]:
donnees01 = donnees.groupby(['pays','annee','mois']).agg({'ca':'sum'})
donnees01.head(20)

In [ ]:
donnees01['CA_a']         = donnees01.groupby(['annee']).ca.transform('sum')
donnees01['palmares_A']  = donnees01.groupby(['annee']).ca.rank(method='dense')
donnees01['CA_p']         = donnees01.groupby(['pays']).ca.transform('sum')
donnees01['palmares_P']  = donnees01.groupby(['pays']).ca.rank(method='dense')
donnees01['palmares']  = donnees01.ca.rank(method='dense')

In [ ]:
donnees01.sort_values(['annee','palmares_A']).head(10)

In [ ]:
donnees01.sort_values(['palmares_A']).head(10)

In [ ]:
donnees01.sort_values(['pays','palmares_P']).head(10)

In [ ]:
donnees01.sort_values(['palmares_P']).head(25)

In [ ]:
donnees01.sort_values(['palmares']).head(25)

## Le palmarès (dense_rank)

In [ ]:
query = """
SELECT
    NOM_PRODUIT, NO_FOURNISSEUR NF, UNITES_STOCK US
FROM PRODUITS 
WHERE CODE_CATEGORIE = 1
"""
donnees01 = pd.read_sql_query(query, connection) #df.to_sql('nom', con=engine,if_exists='replace')
donnees01

In [ ]:
donnees01['RN']  = donnees01.us.rank(method='first')
donnees01['DR']  = donnees01.us.rank(method='dense')
donnees01['AR']  = donnees01.us.rank()
donnees01['NT']  = (donnees01.us.rank(pct=True)*100).round()

In [ ]:
donnees01.sort_values('us')

## La clause de fenêtrage

In [ ]:
donnees02 = donnees.groupby(['pays','annee','mois']).agg({'ca':'sum'})

In [ ]:
donnees02.head()

In [ ]:
donnees02['R3l'] = donnees02.ca.rolling(3).sum()
donnees02['R3lC'] = donnees02.ca.rolling(3,center=True).sum()
donnees02['E3l'] = donnees02.ca.expanding().sum()

In [ ]:
donnees02.head(20)